# Gráfico de Transformação - Auto Featuring - Experimento

Este é um componente que utiliza uma técnica para explorar as possíveis soluções orientadas por um gráfico para executar a AutoFeaturing. <br>

Este notebook apresenta:
- como utilizar o [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Transformer:
    
    def __init__(self):
        # Available transformations
        self.av_transformations = {"numeric" : ["sin", "cos", "tan", "square"],
                                "grouped": ["mean", "median", "std", "sum"],
                                "time": ["second", "minute", "hour", "day", "month", "year", 'dayofweek']}
    
    
    def apply_numeric(self, data, column):

        trans, col = column.split('---')

        np_func = getattr(np, trans)

        return data[col].apply(lambda x: np_func(x))


    def apply_timely(self, data, column):

        trans, col = column.split('---')

        data[col] = data[col].astype(str)
        data[col] = pd.to_datetime(data[col], infer_datetime_format=True)

        if trans == 'dayofweek' or trans == 'weekday':
            return data[col].dt.dayofweek
        else:
            return data[col].apply(lambda x : getattr(x, trans))


    def is_number_repl_isdigit(self, s):
        """
        Returns True ifstring is a number.
        """
        return s.replace('.','',1).isdigit()


    def apply_grouped(self, data, trans_applied, column):

        new_data = data.copy()
        distinct_labels = trans_applied[column]

        new_data['temp_grouping_column'] = 0

        for label in distinct_labels:
            # Convert to original type
            c_label = label
            if self.is_number_repl_isdigit(label):
                if label.isdigit():
                    c_label = int(label)
                else:
                    c_label = float(label)

            new_data.loc[new_data[column.split('---')[2]] == c_label, 'temp_grouping_column'] = distinct_labels[label]

        # Verify if a label was not in the distinct labels
        if 0 in new_data['temp_grouping_column'].unique():
            new_data['temp_grouping_column'] = new_data[new_data['temp_grouping_column'] == 0][column.split('---')[1]]

        return new_data['temp_grouping_column']
    
    
    def transform(self, data, trans_applied, column):
    
        if trans_applied[column] == "":

            if column.split('---')[0] in self.av_transformations['numeric']:
                data[column] = self.apply_numeric(data, column)
            else:
                data[column] = self.apply_timely(data, column)

        else:
            data[column] = self.apply_grouped(data, trans_applied, column)

        return data


class Model(object):
    def __init__(self, dataset: str = None, target: str = None):
        # Carrega artefatos: estimador, etc
        model = load_model()
        self.features = model["features"]
        self.transformations = model["transformations"]
        self.features_names_training = model["columns"]
        
    def class_names(self):
        return self.features_names_training

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        data = pd.DataFrame(data=X, columns=self.features_names_training)
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            data = df[self.features_names_training]
        
        t = Transformer()
        for column in self.transformations:
            data = t.transform(data, self.transformations, column)
        
        return data.to_numpy()

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "crim",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "zn",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "indus",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "chas",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1.0]
        },
        {
            "name": "nox",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1.0]
        },
        {
            "name": "rm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 10.0]
        },
        {
            "name": "age",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "dis",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "rad",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 24.0]
        },
        {
            "name": "tax",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1000.0]
        },
        {
            "name": "ptratio",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "black",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1000.0]
        },
        {
            "name": "lstat",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "medv",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        }
    ],
    "targets": []
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")